In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
os.chdir("..")

In [2]:
LOGS_ROOT_DIR = Path("lightning_logs")

In [39]:
logs_raw_paths = list(LOGS_ROOT_DIR.rglob("**/*.csv"))
logs_raw_paths

[PosixPath('lightning_logs/MNISTClassificationModule_MNISTProvider/raw_results.csv'),
 PosixPath('lightning_logs/MNISTIdentityModule_MNISTProvider/raw_results.csv'),
 PosixPath('lightning_logs/MNISTMappingModule_MNISTProvider/raw_results.csv')]

In [47]:
logs_path = logs_raw_paths[0]
logs_path.parts[1]

'MNISTClassificationModule_MNISTProvider'

In [52]:
ACCURACY = "Classification" in logs_path.parts[1]
LOSS_METRIC = "CE" if ACCURACY else "MSE"


df = pd.read_csv(logs_path)
df.head()

,accuracy,val_loss,experiment,model,experiment_name,hidden_size,num_hidden,seed
0,0.970186,0.079579,MNISTClassificationModule_MNISTProvider,DepthwiseAutomorpherNet,20240812_h128_one_hidden_s79,128,one_hidden,79
1,0.919628,0.276078,MNISTClassificationModule_MNISTProvider,DepthwiseAutomorpherNet,20240812_h128_identity_s19,128,identity,19
2,0.919152,0.276468,MNISTClassificationModule_MNISTProvider,DepthwiseAutomorpherNet,20240812_h128_identity_s17,128,identity,17
3,0.970218,0.077968,MNISTClassificationModule_MNISTProvider,DepthwiseAutomorpherNet,20240812_h128_one_hidden_s53,128,one_hidden,53
4,0.970206,0.082511,MNISTClassificationModule_MNISTProvider,DepthwiseAutomorpherNet,20240812_h128_one_hidden_s37,128,one_hidden,37


In [53]:
# Remap values of num_hidden
hidden_map = {
    "identity": 0,
    "one_hidden": 1,
    "three_hidden": 3
}
df["num_hidden"] = df["num_hidden"].apply(lambda x: hidden_map[x])

# Remap values of model
models_map = {
    "AutomorpherNet": "MorpherNet",
    "DepthwiseAutomorpherNet": "MorpherNetv2"
}
df["model"] = df["model"].apply(lambda x: models_map[x] if x in models_map else x)

# Rename columns
cols_to_rename = ["val_loss"]
if ACCURACY:
    cols_to_rename.append("accuracy")
for col in cols_to_rename:
    df[f"{col}_mean"] = df[col].copy()
    df = df.rename(columns={col:f"{col}_stddev"})
df = df.rename(columns={"hidden_size": "hidden size", "num_hidden":"# hidden"})

In [54]:
GROUPBY_COLUMNS = ["hidden size", "# hidden", "model"]

aggregations_dict = {
            "val_loss_mean": np.mean, 
            "val_loss_stddev": np.std
        }
if ACCURACY:
    aggregations_dict["accuracy_mean"] = np.mean
    aggregations_dict["accuracy_stddev"] = np.std

aggregated = df.groupby(GROUPBY_COLUMNS)\
    .agg(aggregations_dict)

/var/folders/j_/gy0508_n2w36wpr12gx3tz2m0000gn/T/ipykernel_82608/92589555.py:12: FutureWarning: The provided callable <function mean at 0x1053c4430> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg(aggregations_dict)
/var/folders/j_/gy0508_n2w36wpr12gx3tz2m0000gn/T/ipykernel_82608/92589555.py:12: FutureWarning: The provided callable <function std at 0x1053c4550> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(aggregations_dict)
/var/folders/j_/gy0508_n2w36wpr12gx3tz2m0000gn/T/ipykernel_82608/92589555.py:12: FutureWarning: The provided callable <function mean at 0x1053c4430> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mea

In [55]:
# val loss
aggregated[LOSS_METRIC] = aggregated["val_loss_mean"].apply(lambda x: f"{x:.4f}") \
    + " "\
    + aggregated["val_loss_stddev"].apply(lambda x: f"({x:.4f})")
aggregated = aggregated.drop(columns=["val_loss_mean", "val_loss_stddev"])

In [56]:
# accuracy
if ACCURACY:
    aggregated["accuracy"] = aggregated["accuracy_mean"].apply(lambda x: f"{x*100:.2f}%") \
        + " "\
        + aggregated["accuracy_stddev"].apply(lambda x: f"({x*100:.2f}%)")
    aggregated = aggregated.drop(columns=["accuracy_mean", "accuracy_stddev"])

In [57]:
aggregated

CE        accuracy
hidden size # hidden model                                        
128         0        BaselineNet   0.0718 (0.0006)  97.58% (0.05%)
                     MorpherNet    0.2737 (0.0013)  92.07% (0.03%)
                     MorpherNetv2  0.2761 (0.0006)  91.97% (0.04%)
            1        BaselineNet   0.0754 (0.0023)  97.66% (0.14%)
                     MorpherNet    0.0876 (0.0031)  96.85% (0.04%)
                     MorpherNetv2  0.0802 (0.0032)  97.05% (0.06%)
            3        BaselineNet   0.0811 (0.0029)  97.59% (0.02%)
                     MorpherNet    0.0861 (0.0032)  97.07% (0.05%)
                     MorpherNetv2  0.0835 (0.0013)  97.18% (0.04%)

In [13]:
print(aggregated.to_latex())

\begin{tabular}{llll}
\toprule
 &  &  & MSE \\
hidden size & # hidden & model &  \\
\midrule
\multirow[t]{9}{*}{128} & \multirow[t]{3}{*}{0} & BaselineNet & 0.0149 (0.0002) \\
 &  & MorpherNet & 0.0911 (0.0238) \\
 &  & MorpherNetv2 & 0.0813 (0.0001) \\
\cline{2-4}
 & \multirow[t]{3}{*}{1} & BaselineNet & 0.0070 (0.0003) \\
 &  & MorpherNet & 0.0149 (0.0053) \\
 &  & MorpherNetv2 & 0.0087 (0.0002) \\
\cline{2-4}
 & \multirow[t]{3}{*}{3} & BaselineNet & 0.0055 (0.0003) \\
 &  & MorpherNet & 0.0107 (0.0101) \\
 &  & MorpherNetv2 & 0.0056 (0.0001) \\
\cline{1-4} \cline{2-4}
\bottomrule
\end{tabular}

